f11b_bfi_good
f11b_bfi_nad
f11b_nfi_neutral
f11b_sd3_good
f11b_sd3_nad
f11b_sd3_neutral
f7b
l70b
l8b
cs35
gpt

In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from collections import Counter
import re
from scipy.stats import mode

### Score calculating function

In [144]:
def bfi_calc(df):
    # Define the scoring keys
    scoring_key = {
        'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
        'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
        'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
        'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
        'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
    }
    
    
    # Function to calculate trait score
    def calculate_trait_score(persona_data, trait_items):
        scores = []
        for item in trait_items:
            
            score = persona_data.loc[persona_data['ItemNum'] == item, 'mode'].iloc[0]
            scores.append(score)
        return np.mean(scores)
    
    # Group by persona and calculate scores
    results = []
    for (persona, source), group in df.groupby(['Name', 'source']): 
    
        scores = {}
        for trait, items in scoring_key.items():
            scores[trait] = calculate_trait_score(group, items)
        results.append({'Name': persona,'source':source, **scores})
    
    # Create a dataframe with the results
    results_df = pd.DataFrame(results)
    results_df=results_df.round(3)
    # Display the results
    print(results_df.shape)
    print(results_df)

    return results_df  

In [3]:
def bfi10_calc(df):
    # Define the scoring keys
    scoring_key = {
        'Extraversion': [1, 6],
        'Agreeableness': [2, 7],
        'Conscientiousness': [3, 8],
        'Neuroticism': [4, 9],
        'Openness': [5, 10]
    }
    
    
    # Function to calculate trait score
    def calculate_trait_score(persona_data, trait_items):
        scores = []
        for item in trait_items:
            
            score = persona_data.loc[persona_data['ItemNum'] == item, 'mode'].iloc[0]
            scores.append(score)
        return np.mean(scores)
    
    # Group by persona and calculate scores
    results = []
    for (persona, source), group in df.groupby(['Name', 'source']): 
    
        scores = {}
        for trait, items in scoring_key.items():
            scores[trait] = calculate_trait_score(group, items)
        results.append({'Name': persona,'source':source, **scores})
    
    # Create a dataframe with the results
    results_df = pd.DataFrame(results)
    results_df=results_df.round(3)
    # Display the results
    print(results_df.shape)
    print(results_df)

    return results_df  

In [145]:

def sd3_calc2(df):
    # Define the scoring keys
    scoring_key = {
        'Machiavellianism': [1, 2, 3, 4, 5, 6, 7, 8, 9],
        'Narcissism': [10, 11, 12, 13, 14, 15, 16, 17, 18],
        'Psychopathy': [19, 20, 21, 22, 23, 24, 25, 26, 27]
    }
    
    def calculate_trait_score(persona_data, trait_items):
        scores = []
        for item in trait_items:
            # Get the adjusted_score for the given item
            score = persona_data.loc[persona_data['ItemNum'] == item, 'mode'].iloc[0]
            scores.append(score)
        return np.mean(scores)
    
    # Group by Name and source and calculate scores
    results = []
    for (name, source), group in df.groupby(['Name', 'source']):
        scores = {'Name': name, 'source': source}
        for trait, items in scoring_key.items():
            scores[trait] = calculate_trait_score(group, items)
        results.append(scores)
    
    # Create a dataframe with the results
    results_df = pd.DataFrame(results)
    results_df = results_df.round(3)
    
    # Display the results
    print(results_df)
    return results_df


### Adjusting for reverse scored items

In [140]:
counthit=0
# Define reverse-scored items

# Function to adjust scores
def adjust_score(row):
    global counthit
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
    if row['ItemNum'] in reverse_scored:
        #print("hit")
        counthit=counthit+1
        return 6 - row['Answer_numeric']
    return row['Answer_numeric']


In [141]:
count2=0
# Define reverse-scored items
reverse_scored_bfi10 = [1,7,3,4,5]
# Function to adjust scores
def adjust_10bfi_score(row):
    global count2
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    if row['ItemNum'] in reverse_scored_bfi10:
        #print("hit")
        count2=count2+1
        return 6 - row['Answer_numeric']
    return row['Answer_numeric']

In [142]:
count3=0
# Define reverse-scored items

# Function to adjust scores for sd3
def adjust_score2(row):
    global count3
    reverse_scored_sd3 = [11,15,17,20,25]
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    if row['ItemNum'] in reverse_scored_sd3:
        #print("hit")
        count3=count3+1
        return 6 - row['Answer_numeric']
    
    return row['Answer_numeric']


#### Extracting number value

In [143]:
def extract_number(text):
    if pd.isna(text):
        return None
    if isinstance(text, (int, float)):
        if(text in [1,2,3,4,5]):
            return int(text)
        else:
            return None
    match = re.fullmatch(r'^[^1-5]*([1-5])[^1-5]*$', text)
    return int(match.group(1)) if match else None

### Falcon 11 B

##### bad

In [268]:
df_m1_b=pd.read_csv('./F11B_res/results_bfi_badp1.csv')
df_m1_b=pd.read_csv('./F11B_res/results_bfi_badp2.csv')

In [269]:
df_m1_listb=[df_m1_b,df_m1_b]
df_m1_comb_b = pd.concat(df_m1_listb, ignore_index=True)

In [270]:
df_m1_comb_b.shape

(236016, 5)

In [271]:
# Apply the function to the 'Answer' column
df_m1_comb_b['Answer'] = df_m1_comb_b['Answer'].apply(extract_number)

df_m1_comb_b['Answer_numeric'] = pd.to_numeric(df_m1_comb_b['Answer'], errors='coerce')

# Identify non-numeric values
df_m1_comb_b['Is_non_numeric'] = df_m1_comb_b['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_m1_comb_b.groupby('Itemnum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


Itemnum
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
Name: Is_non_numeric, dtype: int64


In [272]:
nan_count = df_m1_comb_b['Answer_numeric'].isna().sum()
nan_count

np.int64(0)

In [ ]:
#If there is a guardrail for a particular item, we replace tha value with 3
os.sleep(10)
#df_m1_comb_g.loc[(df_m1_comb_g['ItemNum'] == 4) & (df_m1_comb_g['Answer_numeric'].isna()), 'Answer'] = 3

In [273]:
df_m1_comb_b.head

<bound method NDFrame.head of                    Name     source  \
0       Osama bin Laden  Terrorist   
1       Osama bin Laden  Terrorist   
2       Osama bin Laden  Terrorist   
3       Osama bin Laden  Terrorist   
4       Osama bin Laden  Terrorist   
...                 ...        ...   
236011   Manuel Noriega   Dictator   
236012   Manuel Noriega   Dictator   
236013   Manuel Noriega   Dictator   
236014   Manuel Noriega   Dictator   
236015   Manuel Noriega   Dictator   

                                                 Item  Itemnum  Answer  \
0                                        Is talkative        1       1   
1                     Tends to find fault with others        2       3   
2                                 Does a thorough job        3       5   
3                                  Is depressed, blue        4       1   
4                Is original, comes up with new ideas        5       1   
...                                               ...      ...     ..

In [274]:
df_m1_comb_b['adjusted_score'] = df_m1_comb_b.apply(adjust_score,axis=1)

In [277]:
df_m1_comb_b.head

<bound method NDFrame.head of                    Name     source  \
0       Osama bin Laden  Terrorist   
1       Osama bin Laden  Terrorist   
2       Osama bin Laden  Terrorist   
3       Osama bin Laden  Terrorist   
4       Osama bin Laden  Terrorist   
...                 ...        ...   
236011   Manuel Noriega   Dictator   
236012   Manuel Noriega   Dictator   
236013   Manuel Noriega   Dictator   
236014   Manuel Noriega   Dictator   
236015   Manuel Noriega   Dictator   

                                                 Item  Itemnum  Answer  \
0                                        Is talkative        1       1   
1                     Tends to find fault with others        2       3   
2                                 Does a thorough job        3       5   
3                                  Is depressed, blue        4       1   
4                Is original, comes up with new ideas        5       1   
...                                               ...      ...     ..

In [278]:
import pandas as pd
from scipy.stats import mode

# Group by the specified columns and calculate mean, std, and mode
avg_score = df_m1_comb_b.groupby(['Name', 'Itemnum', 'source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)

(19668, 6)


In [279]:
avg_score

,Name,Itemnum,source,mean,std,mode
0,2 Pistols,1,Druglord,2.500000,1.882938,1
1,2 Pistols,2,Druglord,1.666667,1.154701,1
2,2 Pistols,3,Druglord,1.833333,1.114641,1
3,2 Pistols,4,Druglord,2.500000,1.167748,2
4,2 Pistols,5,Druglord,1.833333,1.527525,1
...,...,...,...,...,...,...
19663,Ángel Maturino Reséndiz,40,SerialKiller,2.166667,1.749459,1
19664,Ángel Maturino Reséndiz,41,SerialKiller,3.000000,1.809068,1
19665,Ángel Maturino Reséndiz,42,SerialKiller,2.666667,1.775251,1
19666,Ángel Maturino Reséndiz,43,SerialKiller,1.666667,1.556998,1


In [89]:
avg_m1_b.head(27)

,Name,Itemnum,source,adjusted_score
0,2 Pistols,1,Druglord,2.000000
1,2 Pistols,2,Druglord,3.833333
2,2 Pistols,3,Druglord,2.666667
3,2 Pistols,4,Druglord,2.166667
4,2 Pistols,5,Druglord,3.333333
5,2 Pistols,6,Druglord,3.000000
6,2 Pistols,7,Druglord,2.166667
7,2 Pistols,8,Druglord,2.500000
8,2 Pistols,9,Druglord,2.333333
9,2 Pistols,10,Druglord,2.833333


In [280]:
avg_m1_b=avg_score

In [281]:
avg_score['std'].mean()

np.float64(1.619563614445)

In [282]:
df_drop_m1_b=avg_m1_b.loc[avg_m1_b['mode'].isna()]['Name']
print(df_drop_m1_b)
df_filt_m1_b = avg_m1_b[~avg_m1_b['Name'].isin(df_drop_m1_b)]

df_filt_m1_b = df_filt_m1_b.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_m1_b)}")
print(f"Number of rows after filtering: {len(df_filt_m1_b)}")
print(f"Number of rows removed: {len(avg_m1_b) - len(df_filt_m1_b)}")

Series([], Name: Name, dtype: object)
Original number of rows: 19668
Number of rows after filtering: 19668
Number of rows removed: 0


In [283]:
result_m1_bfi_bm=bfi_calc(df_filt_m1_b)

(447, 7)
                        Name        source  Extraversion  Agreeableness  \
0                  2 Pistols      Druglord         1.875          1.444   
1         Abdelhamid Abaaoud     Terrorist         2.000          2.333   
2            Abdolmalek Rigi     Terrorist         2.000          1.444   
3         Abdulaziz al-Omari     Terrorist         1.500          1.000   
4    Abdullah Ahmed Abdullah     Terrorist         2.625          2.222   
..                       ...           ...           ...            ...   
442               Yevno Azef     Terrorist         2.000          1.889   
443           Yoo Young-chul  SerialKiller         2.000          1.444   
444            Yoshio Kodama      Druglord         2.000          2.333   
445            Zodiac Killer  SerialKiller         2.250          1.889   
446  Ángel Maturino Reséndiz  SerialKiller         2.000          2.333   

     Conscientiousness  Neuroticism  Openness  
0                1.333        1.375       

In [284]:
result_m1_bfi_bm.to_csv('./test_scores/mode/BFI/result_f11b_bfi_bm.csv')

##### neutral

In [336]:
df_m1_n=pd.read_csv('./F11B_res/results_sd3_goodp1.csv')
df_m1_n=pd.read_csv('./F11B_res/results_sd3_goodp2.csv')

In [337]:
df_m1_listb=[df_m1_n,df_m1_n]
df_m1_comb_n = pd.concat(df_m1_listb, ignore_index=True)

In [338]:
df_m1_comb_n.shape

(138348, 5)

In [339]:
# Apply the function to the 'Answer' column
df_m1_comb_n['Answer'] = df_m1_comb_n['Answer'].apply(extract_number)

df_m1_comb_n['Answer_numeric'] = pd.to_numeric(df_m1_comb_n['Answer'], errors='coerce')

# Identify non-numeric values
df_m1_comb_n['Is_non_numeric'] = df_m1_comb_n['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_m1_comb_n.groupby('Itemnum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


Itemnum
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
Name: Is_non_numeric, dtype: int64


In [340]:
nan_count = df_m1_comb_n['Answer_numeric'].isna().sum()
nan_count

np.int64(0)

In [341]:
df_m1_comb_n.head()

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric
0,Barack Obama,NPP,It's not wise to tell your secrets,1,1,1,False
1,Barack Obama,NPP,I like to use clever manipulation to get my way,2,5,5,False
2,Barack Obama,NPP,"Whatever it takes, you must get the important ...",3,5,5,False
3,Barack Obama,NPP,Avoid direct conflict with others because they...,4,1,1,False
4,Barack Obama,NPP,It's wise to keep track of information that yo...,5,2,2,False


In [342]:
df_m1_comb_n['adjusted_score'] = df_m1_comb_n.apply(adjust_score2,axis=1)

In [292]:
print(count3)

59520


In [343]:
df_m1_comb_n

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric,adjusted_score
0,Barack Obama,NPP,It's not wise to tell your secrets,1,1,1,False,1
1,Barack Obama,NPP,I like to use clever manipulation to get my way,2,5,5,False,5
2,Barack Obama,NPP,"Whatever it takes, you must get the important ...",3,5,5,False,5
3,Barack Obama,NPP,Avoid direct conflict with others because they...,4,1,1,False,1
4,Barack Obama,NPP,It's wise to keep track of information that yo...,5,2,2,False,2
...,...,...,...,...,...,...,...,...
138343,Gerhard Fischer,GPP,It's true that I can be mean to others,23,4,4,False,4
138344,Gerhard Fischer,GPP,People who mess with me always regret it,24,1,1,False,1
138345,Gerhard Fischer,GPP,I have never gotten into trouble with the law,25,5,5,False,1
138346,Gerhard Fischer,GPP,I enjoy having sex with people I hardly know,26,5,5,False,5


In [344]:
import pandas as pd
from scipy.stats import mode

# Group by the specified columns and calculate mean, std, and mode
avg_score = df_m1_comb_n.groupby(['Name', 'Itemnum', 'source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)


(11529, 6)


In [345]:
avg_m1_n=avg_score

In [346]:
avg_score['std'].mean()

np.float64(1.5857034652782325)

In [347]:
avg_m1_n.shape

(11529, 6)

In [348]:
avg_m1_n

,Name,Itemnum,source,mean,std,mode
0,A. T. Ariyaratne,1,RM,3.000000,1.477098,4
1,A. T. Ariyaratne,2,RM,2.666667,1.775251,1
2,A. T. Ariyaratne,3,RM,3.666667,1.435481,4
3,A. T. Ariyaratne,4,RM,3.166667,1.642245,4
4,A. T. Ariyaratne,5,RM,3.166667,1.749459,1
...,...,...,...,...,...,...
11524,Óscar Arias Sánchez,23,NPP,2.166667,1.527525,1
11525,Óscar Arias Sánchez,24,NPP,4.000000,1.044466,4
11526,Óscar Arias Sánchez,25,NPP,1.833333,1.527525,1
11527,Óscar Arias Sánchez,26,NPP,2.166667,1.749459,1


In [349]:
df_drop_m1_n=avg_m1_n.loc[avg_m1_n['mode'].isna()]['Name']
print(df_drop_m1_n)
df_filt_m1_n = avg_m1_n[~avg_m1_n['Name'].isin(df_drop_m1_n)]

df_filt_m1_n = df_filt_m1_n.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_m1_n)}")
print(f"Number of rows after filtering: {len(df_filt_m1_n)}")
print(f"Number of rows removed: {len(avg_m1_n) - len(df_filt_m1_n)}")

Series([], Name: Name, dtype: object)
Original number of rows: 11529
Number of rows after filtering: 11529
Number of rows removed: 0


In [350]:
result_m1_bfi_gm=sd3_calc2(df_filt_m1_n)

                       Name source  Machiavellianism  Narcissism  Psychopathy
0          A. T. Ariyaratne     RM             2.889       2.222        2.111
1         A.H.M. Noman Khan     RM             2.444       1.556        2.111
2             Abdon Nababan     RM             2.222       3.889        3.333
3       Abdul Razak Hussein     RM             2.111       2.778        2.222
4        Abdul Samad Ismail     RM             1.556       3.222        1.889
..                      ...    ...               ...         ...          ...
422         Zacarias Sarian     RM             2.222       3.000        1.778
423     Zafrullah Chowdhury     RM             3.000       2.000        2.556
424  Zakiah Hanum Abd Hamid     RM             1.556       2.333        1.667
425           Élie Ducommun    NPP             1.667       2.444        2.000
426     Óscar Arias Sánchez    NPP             2.111       3.000        2.000

[427 rows x 5 columns]


In [351]:
result_m1_bfi_gm.to_csv('./test_scores/mode/SD3/result_f11b_sd3_gm.csv')

#### good

In [317]:
df_m1_n=pd.read_csv('./F11B_res/results_sd3_goodp1.csv')
df_m1_n=pd.read_csv('./F11B_res/results_sd3_goodp2.csv')

In [318]:
df_m1_listb=[df_m1_n,df_m1_n]
df_m1_comb_n = pd.concat(df_m1_listb, ignore_index=True)

In [319]:
df_m1_comb_n.shape

(138348, 5)

In [320]:
# Apply the function to the 'Answer' column
df_m1_comb_n['Answer'] = df_m1_comb_n['Answer'].apply(extract_number)

df_m1_comb_n['Answer_numeric'] = pd.to_numeric(df_m1_comb_n['Answer'], errors='coerce')

# Identify non-numeric values
df_m1_comb_n['Is_non_numeric'] = df_m1_comb_n['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_m1_comb_n.groupby('Itemnum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


Itemnum
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
Name: Is_non_numeric, dtype: int64


In [ ]:
#If there is a guardrail for a particular item, we replace tha value with 3
os.sleep(10)
#df_m1_comb_g.loc[(df_m1_comb_g['ItemNum'] == 4) & (df_m1_comb_g['Answer_numeric'].isna()), 'Answer'] = 3

In [13]:
df_m1_comb_n.head()

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric
0,Barack Obama,NPP,It's not wise to tell your secrets,1,1,1,False
1,Barack Obama,NPP,I like to use clever manipulation to get my way,2,5,5,False
2,Barack Obama,NPP,"Whatever it takes, you must get the important ...",3,5,5,False
3,Barack Obama,NPP,Avoid direct conflict with others because they...,4,1,1,False
4,Barack Obama,NPP,It's wise to keep track of information that yo...,5,2,2,False


In [14]:
df_m1_comb_n['adjusted_score'] = df_m1_comb_n.apply(adjust_score2,axis=1)

In [15]:
print(count3)

25620


In [ ]:
import pandas as pd
from scipy.stats import mode

# Group by the specified columns and calculate mean, std, and mode
avg_score = df_m1_comb_b.groupby(['Name', 'Itemnum', 'source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)
avg_m1_n= df_m1_comb_n.groupby(['Name', 'Itemnum','source'])['adjusted_score'].mean().reset_index()

In [17]:
avg_m1_n.shape

(11529, 4)

In [18]:
avg_m1_n.head(27)

,Name,Itemnum,source,adjusted_score
0,A. T. Ariyaratne,1,RM,3.000000
1,A. T. Ariyaratne,2,RM,2.666667
2,A. T. Ariyaratne,3,RM,3.666667
3,A. T. Ariyaratne,4,RM,3.166667
4,A. T. Ariyaratne,5,RM,3.166667
5,A. T. Ariyaratne,6,RM,2.166667
6,A. T. Ariyaratne,7,RM,3.666667
7,A. T. Ariyaratne,8,RM,3.833333
8,A. T. Ariyaratne,9,RM,2.833333
9,A. T. Ariyaratne,10,RM,3.833333


In [19]:
df_drop_m1_n=avg_m1_n.loc[avg_m1_n['adjusted_score'].isna()]['Name']
print(df_drop_m1_n)
df_filt_m1_n = avg_m1_n[~avg_m1_n['Name'].isin(df_drop_m1_n)]

df_filt_m1_n = df_filt_m1_n.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_m1_n)}")
print(f"Number of rows after filtering: {len(df_filt_m1_n)}")
print(f"Number of rows removed: {len(avg_m1_n) - len(df_filt_m1_n)}")

Series([], Name: Name, dtype: object)
Original number of rows: 11529
Number of rows after filtering: 11529
Number of rows removed: 0


In [20]:
result_m1_sd3_g=sd3_calc2(df_filt_m1_n)

                       Name source  Machiavellianism  Narcissism  Psychopathy
0          A. T. Ariyaratne     RM             3.130       3.037        2.611
1         A.H.M. Noman Khan     RM             2.815       2.889        2.648
2             Abdon Nababan     RM             3.093       3.426        3.185
3       Abdul Razak Hussein     RM             2.907       3.111        2.796
4        Abdul Samad Ismail     RM             2.796       3.352        2.537
..                      ...    ...               ...         ...          ...
422         Zacarias Sarian     RM             3.037       2.852        2.685
423     Zafrullah Chowdhury     RM             3.037       2.981        2.778
424  Zakiah Hanum Abd Hamid     RM             2.537       3.000        2.648
425           Élie Ducommun    NPP             2.500       3.185        2.907
426     Óscar Arias Sánchez    NPP             2.907       3.296        2.667

[427 rows x 5 columns]


In [21]:
result_m1_sd3_g.to_csv('./test_scores/SD3/result_f11b_sd3_g.csv')

### Falcon 7B

In [27]:
df1=pd.read_csv('./F7B_res/results_bfi_neutralp1.csv')  
df2=pd.read_csv('./F7B_res/results_bfi_neutralp2.csv')

In [28]:
df_list=[df1,df2]
df_comb = pd.concat(df_list, ignore_index=True)

In [29]:
df_comb.shape

(287760, 5)

In [30]:
# Apply the function to the 'Answer' column
df_comb['Answer'] = df_comb['Answer'].apply(extract_number)

df_comb['Answer_numeric'] = pd.to_numeric(df_comb['Answer'], errors='coerce')

# Identify non-numeric values
df_comb['Is_non_numeric'] = df_comb['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_comb.groupby('Itemnum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


Itemnum
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
Name: Is_non_numeric, dtype: int64


In [13]:
nan_count = df_comb['Answer_numeric'].isna().sum()
nan_count

np.int64(0)

In [31]:
df_comb

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric
0,Michael Jackson,Actor,Is talkative,1,3,3,False
1,Michael Jackson,Actor,Tends to find fault with others,2,2,2,False
2,Michael Jackson,Actor,Does a thorough job,3,5,5,False
3,Michael Jackson,Actor,"Is depressed, blue",4,5,5,False
4,Michael Jackson,Actor,"Is original, comes up with new ideas",5,3,3,False
...,...,...,...,...,...,...,...
287755,Niall Horan,Singer,"Likes to reflect, play with ideas",40,5,5,False
287756,Niall Horan,Singer,Has few artistic interests,41,5,5,False
287757,Niall Horan,Singer,Likes to cooperate with others,42,3,3,False
287758,Niall Horan,Singer,Is easily distracted,43,4,4,False


In [32]:

#If there is a guardrail for a particular item, we replace tha value with 3
# os.sleep(10)
# df_m1_comb_g.loc[(df_m1_comb_g['ItemNum'] == 4) & (df_m1_comb_g['Answer_numeric'].isna()), 'Answer'] = 3

#print(df_comb.shape)

df_comb['adjusted_score'] = df_comb.apply(adjust_score,axis=1)

print(counthit)



59520


In [35]:
print(df_comb.shape)

(287760, 8)


In [36]:
df_comb

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric,adjusted_score
0,Michael Jackson,Actor,Is talkative,1,3,3,False,3
1,Michael Jackson,Actor,Tends to find fault with others,2,2,2,False,2
2,Michael Jackson,Actor,Does a thorough job,3,5,5,False,5
3,Michael Jackson,Actor,"Is depressed, blue",4,5,5,False,5
4,Michael Jackson,Actor,"Is original, comes up with new ideas",5,3,3,False,3
...,...,...,...,...,...,...,...,...
287755,Niall Horan,Singer,"Likes to reflect, play with ideas",40,5,5,False,5
287756,Niall Horan,Singer,Has few artistic interests,41,5,5,False,5
287757,Niall Horan,Singer,Likes to cooperate with others,42,3,3,False,3
287758,Niall Horan,Singer,Is easily distracted,43,4,4,False,4


In [37]:
import pandas as pd
from scipy.stats import mode

# Group by the specified columns and calculate mean, std, and mode
avg_score = df_comb.groupby(['Name', 'Itemnum', 'source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)



(23980, 6)


In [38]:
avg_score

,Name,Itemnum,source,mean,std,mode
0,50 Cent,1,Entrepreneur,3.250000,1.288057,3
1,50 Cent,2,Entrepreneur,3.416667,1.621354,5
2,50 Cent,3,Entrepreneur,3.916667,1.240112,5
3,50 Cent,4,Entrepreneur,3.916667,1.505042,5
4,50 Cent,5,Entrepreneur,4.250000,1.288057,5
...,...,...,...,...,...,...
23975,Édith Piaf,40,Actor,4.250000,0.866025,5
23976,Édith Piaf,41,Actor,3.083333,1.240112,3
23977,Édith Piaf,42,Actor,4.083333,1.240112,5
23978,Édith Piaf,43,Actor,3.583333,1.240112,4


In [39]:
avg_score['std'].mean()

np.float64(1.23181404342406)

In [40]:
df_drop=avg_score.loc[avg_score['mode'].isna()]['Name']
print(df_drop)
df_filt = avg_score[~avg_score['Name'].isin(df_drop)]

df_filt = df_filt.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_score)}")
print(f"Number of rows after filtering: {len(df_filt)}")
print(f"Number of rows removed: {len(avg_score) - len(df_filt)}")


Series([], Name: Name, dtype: object)
Original number of rows: 23980
Number of rows after filtering: 23980
Number of rows removed: 0


In [41]:
result_sd3_n=bfi_calc(df_filt)
result_sd3_n.to_csv('./test_scores/mode/BFI/result_f7b_bfi_nm.csv')

(545, 7)
                       Name        source  Extraversion  Agreeableness  \
0                   50 Cent  Entrepreneur         4.250          4.222   
1               A. A. Milne        Author         3.500          3.444   
2    Abdelkader El Djezairi     Scientist         3.000          4.111   
3           Abraham Lincoln           HoS         4.250          4.333   
4                Ada Yonath     Scientist         4.000          4.111   
..                      ...           ...           ...            ...   
540     Zbigniew Brzezinski        Author         4.125          3.333   
541                 Zendaya         Actor         4.250          4.333   
542           Zhan Beleniuk       Athlete         4.000          4.000   
543      Zlatan Ibrahimovic       Athlete         4.500          4.556   
544              Édith Piaf         Actor         3.750          4.333   

     Conscientiousness  Neuroticism  Openness  
0                4.778        4.750       3.6  
1     

### LLama 70B

In [236]:
import pandas as pd
import glob
import os

# Correct the path construction
path = r'.\LLamaResult\neutral\bfi'  

# Adjust the glob pattern for .csv files
all_files = glob.glob(os.path.join(path, '*.csv'))

df_list = []
print("Files found:", all_files)

# Iterate over the files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(f"Reading file: {filename}")
    print("df shape is:", df.shape)
    df_list.append(df)

# Combine all DataFrames into one
df_comb = pd.concat(df_list, axis=0, ignore_index=True)

print("Combined DataFrame shape:", df_comb.shape)


Files found: ['.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp1_1.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp1_2.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp1_3.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp2_1.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp2_2.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp2_3.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp3_1.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp3_2.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp3_3.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp4_1.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp4_2.csv', '.\\LLamaResult\\neutral\\bfi\\results_bfi_neutralp4_3.csv']
Reading file: .\LLamaResult\neutral\bfi\results_bfi_neutralp1_1.csv
df shape is: (23980, 5)
Reading file: .\LLamaResult\neutral\bfi\results_bfi_neutralp1_2.csv
df shape is: (23980, 5)
Reading file: .\LLamaResult\neutral\bfi\results_bfi_neutralp1_3.csv
df

In [237]:
df_comb.columns

Index(['Name', 'source', 'Item', 'ItemNum', 'Answer'], dtype='object')

In [238]:
# Apply the function to the 'Answer' column
df_comb['Answer'] = df_comb['Answer'].apply(extract_number)

df_comb['Answer_numeric'] = pd.to_numeric(df_comb['Answer'], errors='coerce')

# Identify non-numeric values
df_comb['Is_non_numeric'] = df_comb['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_comb.groupby('ItemNum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


ItemNum
4     16
1      3
36     1
41     1
5      0
6      0
7      0
8      0
9      0
10     0
2      0
3      0
12     0
11     0
13     0
14     0
17     0
18     0
15     0
16     0
21     0
22     0
23     0
24     0
25     0
26     0
19     0
20     0
28     0
27     0
31     0
29     0
32     0
33     0
34     0
30     0
35     0
37     0
39     0
38     0
40     0
42     0
43     0
44     0
Name: Is_non_numeric, dtype: int64


In [239]:
nan_count = df_comb['Answer_numeric'].isna().sum()
nan_count

np.int64(21)

In [240]:
df_comb.head

<bound method NDFrame.head of                    Name  source  \
0       Michael Jackson   Actor   
1       Michael Jackson   Actor   
2       Michael Jackson   Actor   
3       Michael Jackson   Actor   
4       Michael Jackson   Actor   
...                 ...     ...   
287755      Niall Horan  Singer   
287756      Niall Horan  Singer   
287757      Niall Horan  Singer   
287758      Niall Horan  Singer   
287759      Niall Horan  Singer   

                                                 Item  ItemNum  Answer  \
0                                        Is talkative        1     4.0   
1                     Tends to find fault with others        2     2.0   
2                                 Does a thorough job        3     5.0   
3                                  Is depressed, blue        4     1.0   
4                Is original, comes up with new ideas        5     5.0   
...                                               ...      ...     ...   
287755              Likes to re

In [241]:
df_comb.columns

Index(['Name', 'source', 'Item', 'ItemNum', 'Answer', 'Answer_numeric',
       'Is_non_numeric'],
      dtype='object')

In [242]:

#If there is a guardrail for a particular item, we replace tha value with 3
# os.sleep(10)
df_comb.loc[(df_comb['ItemNum'] == 4) & (df_comb['Answer_numeric'].isna()), 'Answer_numeric'] = 3

#print(df_comb.shape)




In [243]:
print(df_comb.shape)

(287760, 7)


In [244]:
df_comb['adjusted_score'] = df_comb.apply(adjust_score,axis=1)

print(counthit)


440256


In [245]:
df_comb

,Name,source,Item,ItemNum,Answer,Answer_numeric,Is_non_numeric,adjusted_score
0,Michael Jackson,Actor,Is talkative,1,4.0,4.0,False,4.0
1,Michael Jackson,Actor,Tends to find fault with others,2,2.0,2.0,False,4.0
2,Michael Jackson,Actor,Does a thorough job,3,5.0,5.0,False,5.0
3,Michael Jackson,Actor,"Is depressed, blue",4,1.0,1.0,False,1.0
4,Michael Jackson,Actor,"Is original, comes up with new ideas",5,5.0,5.0,False,5.0
...,...,...,...,...,...,...,...,...
287755,Niall Horan,Singer,"Likes to reflect, play with ideas",40,4.0,4.0,False,4.0
287756,Niall Horan,Singer,Has few artistic interests,41,1.0,1.0,False,5.0
287757,Niall Horan,Singer,Likes to cooperate with others,42,5.0,5.0,False,5.0
287758,Niall Horan,Singer,Is easily distracted,43,2.0,2.0,False,4.0


In [246]:
avg_score= df_comb.groupby(['Name', 'ItemNum','source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)




(23980, 6)


In [247]:
avg_score

,Name,ItemNum,source,mean,std,mode
0,50 Cent,1,Entrepreneur,3.416667,1.621354,5.0
1,50 Cent,2,Entrepreneur,2.750000,1.055290,2.0
2,50 Cent,3,Entrepreneur,4.583333,0.514929,5.0
3,50 Cent,4,Entrepreneur,2.250000,0.621582,2.0
4,50 Cent,5,Entrepreneur,4.916667,0.288675,5.0
...,...,...,...,...,...,...
23975,Édith Piaf,40,Actor,3.833333,1.114641,4.0
23976,Édith Piaf,41,Actor,5.000000,0.000000,5.0
23977,Édith Piaf,42,Actor,3.750000,0.866025,4.0
23978,Édith Piaf,43,Actor,2.916667,0.900337,2.0


In [248]:
avg_score['std'].mean()

np.float64(0.6943811895634061)

In [249]:
df_drop=avg_score.loc[avg_score['mode'].isna()]['Name']
print(df_drop)
df_filt = avg_score[~avg_score['Name'].isin(df_drop)]

df_filt = df_filt.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_score)}")
print(f"Number of rows after filtering: {len(df_filt)}")
print(f"Number of rows removed: {len(avg_score) - len(df_filt)}")


Series([], Name: Name, dtype: object)
Original number of rows: 23980
Number of rows after filtering: 23980
Number of rows removed: 0


In [250]:
result_sd3_n=bfi_calc(df_filt)
result_sd3_n.to_csv('./test_scores/mode/BFI/result_l70b_bfi_nm.csv')

(545, 7)
                       Name        source  Extraversion  Agreeableness  \
0                   50 Cent  Entrepreneur         4.625          2.000   
1               A. A. Milne        Author         2.625          3.889   
2    Abdelkader El Djezairi     Scientist         3.750          4.333   
3           Abraham Lincoln           HoS         3.000          4.667   
4                Ada Yonath     Scientist         3.375          4.111   
..                      ...           ...           ...            ...   
540     Zbigniew Brzezinski        Author         3.875          3.556   
541                 Zendaya         Actor         4.250          4.000   
542           Zhan Beleniuk       Athlete         3.250          4.222   
543      Zlatan Ibrahimovic       Athlete         5.000          2.222   
544              Édith Piaf         Actor         3.375          3.444   

     Conscientiousness  Neuroticism  Openness  
0                4.000        2.625       3.5  
1     

### LLama 8B

In [122]:
df1=pd.read_csv('./Llama8b/results/results_bfi_goodp1.csv')
df2=pd.read_csv('./Llama8b/results/results_bfi_goodp2.csv')

In [123]:
df_list=[df1,df2]
df_comb = pd.concat(df_list, ignore_index=True)

In [124]:
df_comb.shape

(225456, 5)

In [125]:
# Apply the function to the 'Answer' column
df_comb['Answer'] = df_comb['Answer'].apply(extract_number)

df_comb['Answer_numeric'] = pd.to_numeric(df_comb['Answer'], errors='coerce')

# Identify non-numeric values
df_comb['Is_non_numeric'] = df_comb['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_comb.groupby('Itemnum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


Itemnum
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
Name: Is_non_numeric, dtype: int64


In [126]:
nan_count = df_comb['Answer_numeric'].isna().sum()
nan_count

np.int64(0)

In [127]:
df_comb.columns

Index(['Name', 'source', 'Item', 'Itemnum', 'Answer', 'Answer_numeric',
       'Is_non_numeric'],
      dtype='object')

In [128]:

#If there is a guardrail for a particular item, we replace tha value with 3
# os.sleep(10)
#df_comb.loc[(df_comb['Itemnum'] == 26) & (df_comb['Answer_numeric'].isna()), 'Answer_numeric'] = 3

#print(df_comb.shape)

df_comb['adjusted_score'] = df_comb.apply(adjust_score,axis=1)

print(counthit)



272448


In [129]:
df_comb

,Name,source,Item,Itemnum,Answer,Answer_numeric,Is_non_numeric,adjusted_score
0,Barack Obama,NPP,Is talkative,1,2,2,False,2
1,Barack Obama,NPP,Tends to find fault with others,2,2,2,False,4
2,Barack Obama,NPP,Does a thorough job,3,5,5,False,5
3,Barack Obama,NPP,"Is depressed, blue",4,3,3,False,3
4,Barack Obama,NPP,"Is original, comes up with new ideas",5,4,4,False,4
...,...,...,...,...,...,...,...,...
225451,Gerhard Fischer,GPP,"Likes to reflect, play with ideas",40,4,4,False,4
225452,Gerhard Fischer,GPP,Has few artistic interests,41,3,3,False,3
225453,Gerhard Fischer,GPP,Likes to cooperate with others,42,4,4,False,4
225454,Gerhard Fischer,GPP,Is easily distracted,43,4,4,False,2


In [130]:
print(df_comb.shape)

(225456, 8)


In [131]:
avg_score= df_comb.groupby(['Name', 'Itemnum','source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)




(18788, 6)


In [132]:
avg_score

,Name,Itemnum,source,mean,std,mode
0,A. T. Ariyaratne,1,RM,3.500000,0.522233,3
1,A. T. Ariyaratne,2,RM,2.750000,0.965307,2
2,A. T. Ariyaratne,3,RM,4.500000,0.522233,4
3,A. T. Ariyaratne,4,RM,2.833333,0.717741,3
4,A. T. Ariyaratne,5,RM,4.083333,0.668558,4
...,...,...,...,...,...,...
18783,Óscar Arias Sánchez,40,NPP,4.416667,0.514929,4
18784,Óscar Arias Sánchez,41,NPP,2.750000,0.753778,2
18785,Óscar Arias Sánchez,42,NPP,4.666667,0.492366,5
18786,Óscar Arias Sánchez,43,NPP,3.083333,0.668558,3


In [133]:
avg_score['std'].mean()

np.float64(0.6622522678689616)

In [134]:
df_drop=avg_score.loc[avg_score['mode'].isna()]['Name']
print(df_drop)
df_filt = avg_score[~avg_score['Name'].isin(df_drop)]

df_filt = df_filt.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_score)}")
print(f"Number of rows after filtering: {len(df_filt)}")
print(f"Number of rows removed: {len(avg_score) - len(df_filt)}")


Series([], Name: Name, dtype: object)
Original number of rows: 18788
Number of rows after filtering: 18788
Number of rows removed: 0


In [135]:
result_sd3_b=bfi_calc(df_filt)
result_sd3_b.to_csv('./test_scores/mode/BFI/result_l8b_bfi_gm.csv')

(427, 7)
                       Name source  Extraversion  Agreeableness  \
0          A. T. Ariyaratne     RM         3.500          3.111   
1         A.H.M. Noman Khan     RM         3.500          3.444   
2             Abdon Nababan     RM         3.375          3.556   
3       Abdul Razak Hussein     RM         3.375          3.778   
4        Abdul Samad Ismail     RM         3.500          3.333   
..                      ...    ...           ...            ...   
422         Zacarias Sarian     RM         3.500          3.556   
423     Zafrullah Chowdhury     RM         3.750          3.667   
424  Zakiah Hanum Abd Hamid     RM         3.500          3.667   
425           Élie Ducommun    NPP         3.125          3.667   
426     Óscar Arias Sánchez    NPP         3.375          4.000   

     Conscientiousness  Neuroticism  Openness  
0                3.556        2.875       3.7  
1                3.556        2.875       3.7  
2                3.444        2.750       

### Claude

In [233]:
import pandas as pd
import glob
import os

# Correct the path construction
path = r'.\claude\results\neutral\bfi'  # Use a raw string to avoid escape character issues

# Adjust the glob pattern for .csv files
all_files = glob.glob(os.path.join(path, '*.csv'))

df_list = []
print("Files found:", all_files)

# Iterate over the files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(f"Reading file: {filename}")
    print("df shape is:", df.shape)
    df_list.append(df)

# Combine all DataFrames into one
df_comb = pd.concat(df_list, axis=0, ignore_index=True)

print("Combined DataFrame shape:", df_comb.shape)


Files found: ['.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp1_1.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp1_2.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp1_3.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp2_1.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp2_2.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp2_3.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp3_1.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp3_2.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp3_3.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp4_1.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp4_2.csv', '.\\claude\\results\\neutral\\bfi\\results_bfi_neutralp4_3.csv']
Reading file: .\claude\results\neutral\bfi\results_bfi_neutralp1_1.csv
df shape is: (4000, 5)
Reading file: .\claude\results\neutral\bfi\results_bfi_neutralp1_2.csv
df shape is: (4000, 5)
Reading file: .\cl

In [234]:
df_comb.columns

Index(['Name', 'source', 'Item', 'ItemNum', 'Answer'], dtype='object')

In [235]:
df_comb['Name'].describe()

count           48000
unique            400
top       Niall Horan
freq              120
Name: Name, dtype: object

In [236]:
# Apply the function to the 'Answer' column
df_comb['Answer'] = df_comb['Answer'].apply(extract_number)

df_comb['Answer_numeric'] = pd.to_numeric(df_comb['Answer'], errors='coerce')

# Identify non-numeric values
df_comb['Is_non_numeric'] = df_comb['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_comb.groupby('ItemNum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


ItemNum
7     1
1     0
2     0
3     0
5     0
4     0
6     0
8     0
9     0
10    0
Name: Is_non_numeric, dtype: int64


In [237]:
nan_count = df_comb['Answer_numeric'].isna().sum()
nan_count

np.int64(1)

In [238]:
df_comb.head

<bound method NDFrame.head of                   Name  source                             Item  ItemNum  \
0      Michael Jackson   Actor                      is reserved        1   
1      Michael Jackson   Actor            is generally trusting        2   
2      Michael Jackson   Actor                 tends to be lazy        3   
3      Michael Jackson   Actor  is relaxed, handles stress well        4   
4      Michael Jackson   Actor       has few artistic interests        5   
...                ...     ...                              ...      ...   
47995      Niall Horan  Singer            is outgoing, sociable        6   
47996      Niall Horan  Singer  tends to find fault with others        7   
47997      Niall Horan  Singer              does a thorough job        8   
47998      Niall Horan  Singer              gets nervous easily        9   
47999      Niall Horan  Singer        has an active imagination       10   

       Answer  Answer_numeric  Is_non_numeric  
0        

In [239]:

#If there is a guardrail for a particular item, we replace tha value with 3
# os.sleep(10)
#df_comb.loc[(df_comb['ItemNum'] == 26) & (df_comb['Answer_numeric'].isna()), 'Answer_numeric'] = 3

#print(df_comb.shape)

df_comb['adjusted_score'] = df_comb.apply(adjust_10bfi_score,axis=1)

print(count3)



168000


In [241]:
print(df_comb.shape)

(48000, 8)


In [197]:
df_comb.loc[df_comb['ItemNum']==26]

,Name,source,Item,ItemNum,Answer,Answer_numeric,Is_non_numeric,adjusted_score
25,Barack Obama,NPP,I enjoy having sex with people I hardly know,26,3.0,3.0,False,3.0
52,Nelson Mandela,NPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
79,Martin Luther King Jr.,NPP,I enjoy having sex with people I hardly know,26,NaN,3.0,True,3.0
106,Mikhail Gorbachev,NPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
133,Mother Teresa,NPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
...,...,...,...,...,...,...,...,...
129490,Václav Havel,GPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
129517,Julius Nyerere,GPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
129544,Coretta Scott King,GPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0
129571,Wahiduddin Khan,GPP,I enjoy having sex with people I hardly know,26,1.0,1.0,False,1.0


In [242]:
df_comb.head

<bound method NDFrame.head of                   Name  source                             Item  ItemNum  \
0      Michael Jackson   Actor                      is reserved        1   
1      Michael Jackson   Actor            is generally trusting        2   
2      Michael Jackson   Actor                 tends to be lazy        3   
3      Michael Jackson   Actor  is relaxed, handles stress well        4   
4      Michael Jackson   Actor       has few artistic interests        5   
...                ...     ...                              ...      ...   
47995      Niall Horan  Singer            is outgoing, sociable        6   
47996      Niall Horan  Singer  tends to find fault with others        7   
47997      Niall Horan  Singer              does a thorough job        8   
47998      Niall Horan  Singer              gets nervous easily        9   
47999      Niall Horan  Singer        has an active imagination       10   

       Answer  Answer_numeric  Is_non_numeric  adjusted_s

In [243]:
avg_score= df_comb.groupby(['Name', 'ItemNum','source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)




(4000, 6)


In [244]:
avg_score

,Name,ItemNum,source,mean,std,mode
0,50 Cent,1,Entrepreneur,3.916667,0.792961,4.0
1,50 Cent,2,Entrepreneur,2.916667,0.792961,3.0
2,50 Cent,3,Entrepreneur,3.333333,0.887625,4.0
3,50 Cent,4,Entrepreneur,1.416667,0.900337,1.0
4,50 Cent,5,Entrepreneur,3.416667,0.668558,4.0
...,...,...,...,...,...,...
3995,Édith Piaf,6,Actor,5.000000,0.000000,5.0
3996,Édith Piaf,7,Actor,3.500000,0.904534,4.0
3997,Édith Piaf,8,Actor,5.000000,0.000000,5.0
3998,Édith Piaf,9,Actor,3.666667,1.073087,4.0


In [245]:
avg_score['std'].mean()

np.float64(0.30193974220089265)

In [246]:
df_drop=avg_score.loc[avg_score['mode'].isna()]['Name']
print(df_drop)
df_filt = avg_score[~avg_score['Name'].isin(df_drop)]

df_filt = df_filt.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_score)}")
print(f"Number of rows after filtering: {len(df_filt)}")
print(f"Number of rows removed: {len(avg_score) - len(df_filt)}")


Series([], Name: Name, dtype: object)
Original number of rows: 4000
Number of rows after filtering: 4000
Number of rows removed: 0


In [247]:
result_bfi_nm=bfi10_calc(df_filt)
result_bfi_nm.to_csv('./test_scores/mode/BFI/result_cs35_bfi_nm.csv')

(400, 7)
                       Name        source  Extraversion  Agreeableness  \
0                   50 Cent  Entrepreneur           4.5            3.5   
1               A. A. Milne        Author           2.5            4.0   
2    Abdelkader El Djezairi     Scientist           4.0            4.0   
3           Abraham Lincoln           HoS           4.0            4.0   
4                Ada Yonath     Scientist           4.0            4.0   
..                      ...           ...           ...            ...   
395             Zack Snyder     Scientist           4.0            4.0   
396     Zbigniew Brzezinski        Author           4.0            3.5   
397                 Zendaya         Actor           4.0            4.0   
398      Zlatan Ibrahimovic       Athlete           5.0            5.0   
399              Édith Piaf         Actor           4.5            4.0   

     Conscientiousness  Neuroticism  Openness  
0                  4.5          1.5       4.0  
1     

### GPT Results

In [145]:
import pandas as pd
import glob
import os

# Correct the path construction
path = r'.\Gptresults\Good\SD3'  # Use a raw string to avoid escape character issues

# Adjust the glob pattern for .csv files
all_files = glob.glob(os.path.join(path, '*.csv'))

df_list = []
print("Files found:", all_files)

# Iterate over the files
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    print(f"Reading file: {filename}")
    print("df shape is:", df.shape)
    df_list.append(df)

# Combine all DataFrames into one
df_comb = pd.concat(df_list, axis=0, ignore_index=True)

print("Combined DataFrame shape:", df_comb.shape)


Files found: ['.\\Gptresults\\Good\\SD3\\results_sd3_goodp1_1.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp1_2.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp1_3.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp2_1.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp2_2.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp2_3.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp3_1.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp3_2.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp3_3.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp4_1.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp4_2.csv', '.\\Gptresults\\Good\\SD3\\results_sd3_goodp4_3.csv']
Reading file: .\Gptresults\Good\SD3\results_sd3_goodp1_1.csv
df shape is: (10800, 5)
Reading file: .\Gptresults\Good\SD3\results_sd3_goodp1_2.csv
df shape is: (10800, 5)
Reading file: .\Gptresults\Good\SD3\results_sd3_goodp1_3.csv
df shape is: (10800, 5)
Reading file: .\Gptresults\Good\SD3\results_sd3_goodp2_1.csv
df shape is: (10800, 5

In [146]:
df_comb.columns

Index(['Name', 'source', 'Item', 'ItemNum', 'Answer'], dtype='object')

In [147]:
df_comb['Name'].describe()

count              129600
unique                400
top       Gerhard Fischer
freq                  324
Name: Name, dtype: object

In [148]:
# Apply the function to the 'Answer' column
df_comb['Answer'] = df_comb['Answer'].apply(extract_number)

df_comb['Answer_numeric'] = pd.to_numeric(df_comb['Answer'], errors='coerce')

# Identify non-numeric values
df_comb['Is_non_numeric'] = df_comb['Answer_numeric'].isna()

# Group by 'Itemnum' and count non-numeric answers
result = df_comb.groupby('ItemNum')['Is_non_numeric'].sum()

# Sort the result in descending order
result_sorted = result.sort_values(ascending=False)

# Display the sorted result
print(result_sorted)


ItemNum
26    66
23     3
10     2
12     1
3      0
6      0
5      0
7      0
4      0
2      0
1      0
11     0
9      0
8      0
13     0
16     0
17     0
14     0
15     0
19     0
18     0
21     0
20     0
22     0
24     0
25     0
27     0
Name: Is_non_numeric, dtype: int64


In [149]:
nan_count = df_comb['Answer_numeric'].isna().sum()
nan_count

np.int64(72)

In [150]:
df_comb.head

<bound method NDFrame.head of                    Name source  \
0          Barack Obama    NPP   
1          Barack Obama    NPP   
2          Barack Obama    NPP   
3          Barack Obama    NPP   
4          Barack Obama    NPP   
...                 ...    ...   
129595  Gerhard Fischer    GPP   
129596  Gerhard Fischer    GPP   
129597  Gerhard Fischer    GPP   
129598  Gerhard Fischer    GPP   
129599  Gerhard Fischer    GPP   

                                                     Item  ItemNum  Answer  \
0                     It's not wise to tell your secrets.        1     4.0   
1        I like to use clever manipulation to get my way.        2     2.0   
2       Whatever it takes, you must get the important ...        3     4.0   
3       Avoid direct conflict with others because they...        4     4.0   
4       It's wise to keep track of information that yo...        5     2.0   
...                                                   ...      ...     ...   
129595         

In [151]:

#If there is a guardrail for a particular item, we replace tha value with 3
# os.sleep(10)
df_comb.loc[(df_comb['ItemNum'] == 26) & (df_comb['Answer_numeric'].isna()), 'Answer_numeric'] = 3

#print(df_comb.shape)

df_comb['adjusted_score'] = df_comb.apply(adjust_score2,axis=1)

print(count3)



96000


In [152]:
print(df_comb.shape)

(129600, 8)


In [153]:
df_comb.head

<bound method NDFrame.head of                    Name source  \
0          Barack Obama    NPP   
1          Barack Obama    NPP   
2          Barack Obama    NPP   
3          Barack Obama    NPP   
4          Barack Obama    NPP   
...                 ...    ...   
129595  Gerhard Fischer    GPP   
129596  Gerhard Fischer    GPP   
129597  Gerhard Fischer    GPP   
129598  Gerhard Fischer    GPP   
129599  Gerhard Fischer    GPP   

                                                     Item  ItemNum  Answer  \
0                     It's not wise to tell your secrets.        1     4.0   
1        I like to use clever manipulation to get my way.        2     2.0   
2       Whatever it takes, you must get the important ...        3     4.0   
3       Avoid direct conflict with others because they...        4     4.0   
4       It's wise to keep track of information that yo...        5     2.0   
...                                                   ...      ...     ...   
129595         

In [125]:
df_comb.loc[df_comb['ItemNum']==26]

,Name,source,Item,ItemNum,Answer,Answer_numeric,Is_non_numeric
25,Osama bin Laden,Terrorist,I enjoy having sex with people I hardly know,26,NaN,NaN,True
52,Abu Bakr al-Baghdadi,Terrorist,I enjoy having sex with people I hardly know,26,NaN,NaN,True
79,Anders Behring Breivik,Terrorist,I enjoy having sex with people I hardly know,26,NaN,NaN,True
106,Stepan Bandera,Terrorist,I enjoy having sex with people I hardly know,26,NaN,NaN,True
133,Nigel Farage,Terrorist,I enjoy having sex with people I hardly know,26,NaN,NaN,True
...,...,...,...,...,...,...,...
129490,Alfredo Stroessner,Dictator,I enjoy having sex with people I hardly know,26,1.0,1.0,False
129517,Fulgencio Batista,Dictator,I enjoy having sex with people I hardly know,26,1.0,1.0,False
129544,Jorge Rafael Videla,Dictator,I enjoy having sex with people I hardly know,26,1.0,1.0,False
129571,Georgios Papadopoulos,Dictator,I enjoy having sex with people I hardly know,26,3.0,3.0,False


In [126]:
df_comb.head

<bound method NDFrame.head of                    Name     source  \
0       Osama bin Laden  Terrorist   
1       Osama bin Laden  Terrorist   
2       Osama bin Laden  Terrorist   
3       Osama bin Laden  Terrorist   
4       Osama bin Laden  Terrorist   
...                 ...        ...   
129595   Manuel Noriega   Dictator   
129596   Manuel Noriega   Dictator   
129597   Manuel Noriega   Dictator   
129598   Manuel Noriega   Dictator   
129599   Manuel Noriega   Dictator   

                                                     Item  ItemNum  Answer  \
0                     It's not wise to tell your secrets.        1     5.0   
1        I like to use clever manipulation to get my way.        2     NaN   
2       Whatever it takes, you must get the important ...        3     NaN   
3       Avoid direct conflict with others because they...        4     4.0   
4       It's wise to keep track of information that yo...        5     4.0   
...                                          

In [154]:
avg_score= df_comb.groupby(['Name', 'ItemNum','source'])['adjusted_score'].agg(
    mean='mean',
    std='std',
    mode=lambda x: mode(x, keepdims=True).mode[0]  # Extract the first mode value
).reset_index()
print(avg_score.shape)



(10800, 6)


In [155]:
avg_score

,Name,ItemNum,source,mean,std,mode
0,A. T. Ariyaratne,1,RM,4.083333,0.514929,4.0
1,A. T. Ariyaratne,2,RM,1.083333,0.288675,1.0
2,A. T. Ariyaratne,3,RM,2.500000,1.000000,2.0
3,A. T. Ariyaratne,4,RM,4.166667,0.717741,4.0
4,A. T. Ariyaratne,5,RM,1.166667,0.389249,1.0
...,...,...,...,...,...,...
10795,Óscar Arias Sánchez,23,NPP,1.250000,0.621582,1.0
10796,Óscar Arias Sánchez,24,NPP,2.083333,0.792961,2.0
10797,Óscar Arias Sánchez,25,NPP,1.500000,0.674200,1.0
10798,Óscar Arias Sánchez,26,NPP,1.500000,0.904534,1.0


In [156]:
avg_score['std'].mean()

np.float64(0.5722255968952276)

In [157]:
df_drop=avg_score.loc[avg_score['mean'].isna()]['Name']
print(df_drop)
df_filt = avg_score[~avg_score['Name'].isin(df_drop)]

df_filt = df_filt.reset_index(drop=True)

# Print some information about the filtering process
print(f"Original number of rows: {len(avg_score)}")
print(f"Number of rows after filtering: {len(df_filt)}")
print(f"Number of rows removed: {len(avg_score) - len(df_filt)}")


Series([], Name: Name, dtype: object)
Original number of rows: 10800
Number of rows after filtering: 10800
Number of rows removed: 0


In [158]:
df_filt.head

<bound method NDFrame.head of                       Name  ItemNum source      mean       std  mode
0         A. T. Ariyaratne        1     RM  4.083333  0.514929   4.0
1         A. T. Ariyaratne        2     RM  1.083333  0.288675   1.0
2         A. T. Ariyaratne        3     RM  2.500000  1.000000   2.0
3         A. T. Ariyaratne        4     RM  4.166667  0.717741   4.0
4         A. T. Ariyaratne        5     RM  1.166667  0.389249   1.0
...                    ...      ...    ...       ...       ...   ...
10795  Óscar Arias Sánchez       23    NPP  1.250000  0.621582   1.0
10796  Óscar Arias Sánchez       24    NPP  2.083333  0.792961   2.0
10797  Óscar Arias Sánchez       25    NPP  1.500000  0.674200   1.0
10798  Óscar Arias Sánchez       26    NPP  1.500000  0.904534   1.0
10799  Óscar Arias Sánchez       27    NPP  1.250000  0.452267   1.0

[10800 rows x 6 columns]>

In [159]:
result_sd3_gm=sd3_calc2(df_filt)
result_sd3_gm.to_csv('./test_scores/mode/SD3/result_gpt4_sd3_gm.csv')

                       Name source  Machiavellianism  Narcissism  Psychopathy
0          A. T. Ariyaratne     RM             2.444       3.778        1.333
1         A.H.M. Noman Khan     RM             2.889       3.444        2.111
2             Abdon Nababan     RM             2.667       3.333        2.222
3       Abdul Razak Hussein     RM             2.889       3.333        1.556
4        Abdul Samad Ismail     RM             2.667       3.222        2.111
..                      ...    ...               ...         ...          ...
395           Yukiharu Miki     RM             2.889       3.333        2.000
396     Zafrullah Chowdhury     RM             2.778       3.333        1.667
397  Zakiah Hanum Abd Hamid     RM             2.333       3.333        2.222
398           Élie Ducommun    NPP             2.556       3.556        1.222
399     Óscar Arias Sánchez    NPP             2.778       3.889        1.222

[400 rows x 5 columns]


result_sd3_g=sd3_calc2(df_filt)
result_sd3_g.to_csv('./result_gpt4_sd3_g.csv')